# CORD-19 Software classification

This jupyter notebook is designated to classify software mentions based on the CORD19 dataset from: https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook.

In [1]:
import json

### Connecting to Jupyter Notebook outputting software mentions 

Thusly, the outcome "df_software_mentions" of the notebook "CORD-19-software-counting-cs5099.ipynb" will be used for classification purposes. 

In [2]:
%%time
%run ./CORD-19-software-counting-cs5099.ipynb
df_software_mentions

<ipython-input-2-477d397756ce>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa['Matches'][i] = int(dfa['Matches'][i]+ dfb['Matches'][j])


Wall time: 46.1 s
Wall time: 1min


,Software,Matches,Change
0,'R',10805,0
1,'SPSS',9227,0
4,'BLAST',6376,+2
2,'GRAPHPAD PRISM',4617,-1
3,'EXCEL',4054,-1
...,...,...,...
994,'DR',69,+104
995,'GRAPHPPI',69,+104
996,'POLYMAKE',69,+104
997,'- PAD PRISM',69,+104


Shift the focus to the column software and creat a column for classification

In [3]:
df_software = df_software_mentions.drop('Matches', 1)
df_software = df_software.drop('Change', 1)
df_software = df_software.reset_index()
df_software = df_software.drop('index', 1)
df_software['Classification'] = "Unclassified"
df_software

,Software,Classification
0,'R',Unclassified
1,'SPSS',Unclassified
2,'BLAST',Unclassified
3,'GRAPHPAD PRISM',Unclassified
4,'EXCEL',Unclassified
...,...,...
890,'DR',Unclassified
891,'GRAPHPPI',Unclassified
892,'POLYMAKE',Unclassified
893,'- PAD PRISM',Unclassified


In [4]:
df_software['Software'] = df_software['Software'].str.replace('\'', '')
df_software

,Software,Classification
0,R,Unclassified
1,SPSS,Unclassified
2,BLAST,Unclassified
3,GRAPHPAD PRISM,Unclassified
4,EXCEL,Unclassified
...,...,...
890,DR,Unclassified
891,GRAPHPPI,Unclassified
892,POLYMAKE,Unclassified
893,- PAD PRISM,Unclassified


In [5]:
result = df_software.to_json(orient='records')
parsed = json.loads(result)
software_json = json.dumps(parsed, indent=4) 
print(software_json)

[
    {
        "Software": "R",
        "Classification": "Unclassified"
    },
    {
        "Software": "SPSS",
        "Classification": "Unclassified"
    },
    {
        "Software": "BLAST",
        "Classification": "Unclassified"
    },
    {
        "Software": "GRAPHPAD PRISM",
        "Classification": "Unclassified"
    },
    {
        "Software": "EXCEL",
        "Classification": "Unclassified"
    },
    {
        "Software": "STATA",
        "Classification": "Unclassified"
    },
    {
        "Software": "SAS",
        "Classification": "Unclassified"
    },
    {
        "Software": "MEGA",
        "Classification": "Unclassified"
    },
    {
        "Software": "IMAGEJ",
        "Classification": "Unclassified"
    },
    {
        "Software": "MATLAB",
        "Classification": "Unclassified"
    },
    {
        "Software": "GOOGLE SCHOLAR",
        "Classification": "Unclassified"
    },
    {
        "Software": "GRAPHPAD",
        "Classification": "Unclassi

In [6]:
df_read_json = pd.read_json(software_json)
print(df_read_json.to_string()) 

                                    Software Classification
0                                          R   Unclassified
1                                       SPSS   Unclassified
2                                      BLAST   Unclassified
3                             GRAPHPAD PRISM   Unclassified
4                                      EXCEL   Unclassified
5                                      STATA   Unclassified
6                                        SAS   Unclassified
7                                       MEGA   Unclassified
8                                     IMAGEJ   Unclassified
9                                     MATLAB   Unclassified
10                            GOOGLE SCHOLAR   Unclassified
11                                  GRAPHPAD   Unclassified
12                                       NET   Unclassified
13                                     PRISM   Unclassified
14                                    SCOPUS   Unclassified
15                                  CLUS

In [9]:
df_json_classifier = pd.read_json('software_classification.json')
df_json_classifier

,Software,Classification
0,R,Category 1
1,SPSS,Category 1
2,BLAST,Category 1
3,GRAPHPAD PRISM,Category 1
4,EXCEL,Category 1
...,...,...
890,DR,Category 1
891,GRAPHPPI,Category 1
892,POLYMAKE,Category 1
893,- PAD PRISM,Category 1
